In [ ]:
import pandas as pd
import sys,os
import matplotlib.pyplot as plt
import utils
import concurrent.futures
import importlib

In [ ]:

target_dir = "my_results/icml_experiments/cifar10/"
ATTACK_RESULTS_PATH = "my_results/icml_experiments/cifar10_attack_results_quick/"

TOTAL_PROCESSES = 128
MAX_MACHINES =  8
MAX_ITERATIONS=10000
STARTING_ITERATION = 0

linestyles = {
    "ZeroSum" : "--"
}

fontsize=20

alpha = 0.1

save_directory = "assets/"



assert TOTAL_PROCESSES%MAX_MACHINES == 0
MAX_PROCESSES = TOTAL_PROCESSES//MAX_MACHINES

In [ ]:
# Load the file list
experiments_dict = utils.get_full_path_dict(target_dir)
experiments_attributes = utils.get_experiments_dict(experiments_dict)

print(experiments_dict)

In [ ]:
machine_folder = 'machine{}'
result_file = '{}_results.json'


def load_data(dir):
    data = pd.DataFrame({})
    for machine in range(MAX_MACHINES):
        for rank in range(MAX_PROCESSES):
            # print(f"Loading results for machine {machine} and rank {rank}.  ",end = "\r")
            uid = rank + machine * MAX_PROCESSES

            file = os.path.join(dir, machine_folder.format(machine), result_file.format(rank))
            tmp_df = pd.read_json(file)
            tmp_df["uid"] = uid # Manually add the uid for further processing                                                                   
            tmp_df["iteration"] = tmp_df.index
            # print(tmp_df)
            tmp_df = tmp_df[tmp_df["iteration"]>=STARTING_ITERATION]
            tmp_df = tmp_df[tmp_df["iteration"]<=MAX_ITERATIONS]
            data = pd.concat([data,tmp_df])
    return data




def load_attack_results(current_experiment_data,experiment_name,attack_results_path):
    expected_file_name = experiment_name + ".csv"
    directories = sorted(os.listdir(attack_results_path))
    
    if expected_file_name not in directories:
        raise ValueError(f"{expected_file_name} was not listed with attack results in {attack_results_path}. Entire directory:\n{directories}")
    
    attacks_df = pd.read_csv(os.path.join(attack_results_path,expected_file_name))
    attacks_df = attacks_df.drop(columns = "Unnamed: 0")
    attacks_df = attacks_df.rename(columns = {"agent":"uid"})
    # print(attacks_df.columns)
    # print(current_experiment_data)
    # print(attacks_df)
    
    res = pd.merge(current_experiment_data, attacks_df,on = ["uid","iteration"],how="outer" )
    # print(res)
    return res



def load_data_element(input_dict,name,filepath,attack_result_path):
    print(f"Loading data from {name}")
    current_results = load_data(filepath).dropna()
    input_dict[name] = load_attack_results(current_results,name,attack_results_path=attack_result_path)
    print(f"Finished loading data from {name}")
    return



loaded_data = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for name,filepath in sorted(experiments_dict.items()):
        executor.submit(load_data_element,loaded_data,name,filepath,ATTACK_RESULTS_PATH)
        # load_data_element(loaded_data,name,filepath,ATTACK_RESULTS_PATH)
        # break
print("Finished loading all data")

In [ ]:
agg_methods = ["mean", "std", "sum","min","max"]

def format_data(data,key):
    usable_data = data.groupby('iteration').agg(agg_methods)
    usable_data.reset_index(inplace=True)

    usable_data.insert(1,"experience_name",key)
    usable_data.insert(2,"number_agents",TOTAL_PROCESSES)

    usable_data.columns = [' '.join(e) if len(e[-1])>0 else e[0] for e in usable_data.columns]

    usable_data.set_index("iteration",inplace=True)
    return usable_data

formatted_data = {}
for name,data in loaded_data.items():
    formatted_data[name] = format_data(data,name)
# formatted_data

In [ ]:
utils.plot_all_experiments(
    formatted_data,
    experiments_attributes,
    experiments_attributes,
    None,
    "test_acc mean",
    save_directory=save_directory
    
)


## Noise cancellation vs no noise cancellation

In [ ]:
importlib.reload(utils)
zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["static"]}

display_attributes ={
    "color":{
        "nonoise":"cyan",
        "2th":"red",
        "4th":"orange",
        "8th": "yellow",
        "16th":"green",
        "32th":"blue",
        "64th":"purple",
    },
    "linestyle":{
        "noselfnoise":"--",
        "selfnoise":"-"
    },
    "linewidth":{
        "nonoise":5,
        "zerosum":1,
    }
}

zerosum_experiments = utils.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    experiments_attributes,
    display_attributes,
    "test_acc mean",
    save_directory=save_directory
    
)



### In the dynamic case:

In [ ]:
importlib.reload(utils)
zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["dynamic"]}

display_attributes ={
    "color":{
        "nonoise":"cyan",
        "2th":"red",
        "4th":"orange",
        "8th": "yellow",
        "16th":"green",
        "32th":"blue",
        "64th":"purple",
    },
    "linestyle":{
        "noselfnoise":"--",
        "selfnoise":"-"
    },
    "linewidth":{
        "nonoise":5,
        "zerosum":1,
    }
}

zerosum_experiments = utils.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    experiments_attributes,
    display_attributes,
    "test_acc mean",
    save_directory=save_directory
    
)

# Attack results
## In the static case:

In [ ]:
importlib.reload(utils)
zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["static"]}

display_attributes ={
    "color":{
        "nonoise":"cyan",
        "2th":"red",
        "4th":"orange",
        "8th": "yellow",
        "16th":"green",
        "32th":"blue",
        "64th":"purple",
    },
    "linestyle":{
        "noselfnoise":"--",
        "selfnoise":"-"
    },
    "linewidth":{
        "nonoise":5,
        "zerosum":1,
    }
}

zerosum_experiments = utils.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    experiments_attributes,
    display_attributes,
    "roc_auc mean",
    save_directory=save_directory,
    figsize=(10,10)
)

## Attack results in the dynamic case

In [ ]:
importlib.reload(utils)
zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["dynamic"]}

display_attributes ={
    "color":{
        "nonoise":"cyan",
        "2th":"red",
        "4th":"orange",
        "8th": "yellow",
        "16th":"green",
        "32th":"blue",
        "64th":"purple",
    },
    "linestyle":{
        "noselfnoise":"--",
        "selfnoise":"-"
    },
    "linewidth":{
        "nonoise":5,
        "zerosum":1,
    }
}

zerosum_experiments = utils.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    experiments_attributes,
    display_attributes,
    "roc_auc mean",
    save_directory=save_directory,
    figsize= (10,10)
)

# Results vs Muffliato

In [ ]:
importlib.reload(utils)
zerosum_attributes = {"variant":["nonoise","zerosum","muffliato"],"topology_type":["static"],"avgsteps": [None,"10avgsteps"], "additional_attribute": [None,"selfnoise"]}

display_attributes ={
    "color":{
        "nonoise":"cyan",
        "2th":"red",
        "4th":"orange",
        "8th": "yellow",
        "16th":"green",
        "32th":"blue",
        "64th":"purple",
    },
    "linestyle":{
        "muffliato":"--",
        "zerosum":"-"
    },
    "linewidth":{
        "nonoise":5,
        "zerosum":1,
    }
}

zerosum_experiments = utils.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    experiments_attributes,
    display_attributes,
    "roc_auc mean",
    save_directory=save_directory,
    figsize=(10,10)
)

In [ ]:
importlib.reload(utils)
zerosum_attributes = {"variant":["nonoise","zerosum","muffliato"],"topology_type":["dynamic"],"avgsteps": [None,"10avgsteps"], "additional_attribute": [None,"selfnoise"]}

display_attributes ={
    "color":{
        "nonoise":"cyan",
        "2th":"red",
        "4th":"orange",
        "8th": "yellow",
        "16th":"green",
        "32th":"blue",
        "64th":"purple",
    },
    "linestyle":{
        "muffliato":"--",
        "zerosum":"-"
    },
    "linewidth":{
        "nonoise":5,
        "zerosum":1,
    }
}

zerosum_experiments = utils.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

utils.plot_all_experiments(
    formatted_data,
    zerosum_experiments,
    experiments_attributes,
    display_attributes,
    "roc_auc mean",
    save_directory=save_directory,
    figsize=(10,10)
)